## A. Upload CSVs to S3 (using boto3)

In [ ]:
# install dependencies or put them in a requirements.txt file

# !pip install sqlalchemy
# !pip install sqlalchemy psycopg2-binary pandas


In [ ]:
import boto3
import os

# Setup
bucket_name = "amdari-etl-bucket"
folder_prefix = "medoptix/raw/"
local_folder = "./medoptix_data"

# AWS credentials assumed to be in ~/.aws/credentials or env vars
s3 = boto3.client("s3")

# Upload all CSVs in folder with ACL: bucket-owner-full-control
for file_name in os.listdir(local_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(local_folder, file_name)
        s3.upload_file(
            Filename=file_path,
            Bucket=bucket_name,
            Key=folder_prefix + file_name, 
            ExtraArgs={'ACL': 'bucket-owner-full-control'}  # <-- THIS IS KEY
        )
        print(f"✅ Uploaded {file_name} to s3://{bucket_name}/{folder_prefix}{file_name} with full access")


## B. Load from S3 → PostgreSQL


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import boto3

s3 = boto3.client("s3")


bucket = "amdari-etl-bucket"
prefix = "medoptix/raw/"

# Files to download
files = ["patients.csv", "clinics.csv", "sessions.csv", "feedback.csv", "dropout_flags.csv"]

# Download files
for file in files:
    s3.download_file(bucket, prefix + file, file)
    print(f"⬇️ Downloaded {file} from S3")



# Data Modelling & Defining Foreign Key relationships


- patients (PK: patient_id)
- sessions (PK: session_id, FK: patient_id)
- feedback (PK: feedback_id, FK: session_id)


TASK 1 - Model and Define the relationship for the remaining set of dataset
 - clinics
 - dropout_flags
 - intervention.csv

TASK 2 - Create Schema for these dataset and upload then into postgres

## C. Upload Data → PostgreSQL


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables (recommended for security)
load_dotenv()

def get_db_engine():
    """Create and return a SQLAlchemy engine with proper connection string"""
    # Construct connection string from environment variables
    db_url = (
        f"postgresql://{''}:{''}@"
        f"{''}:{''}/{''}?"
    )
    return create_engine(db_url)

def upload_data():
    # Step 1: Read CSV files
    patients = pd.read_csv("C:/Users/Muham/Downloads/Medoptix_Demo/medoptix_data/processed/patients.csv")
    sessions = pd.read_csv("C:/Users/Muham/Downloads/Medoptix_Demo/medoptix_data/processed/sessions.csv")
    feedback = pd.read_csv("C:/Users/Muham/Downloads/Medoptix_Demo/medoptix_data/processed/feedback.csv")

    # Step 2: Create database engine
    engine = get_db_engine()

    # Step 3: Upload in referential order with error handling
    with engine.begin() as connection:  # Automatically handles transactions
        # Chunk size for large datasets (adjust as needed)
        chunk_size = 1000

        # Upload patients table
        patients.to_sql(
            "patients", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'  # Faster for bulk inserts
        )

        # Upload sessions table
        sessions.to_sql(
            "sessions", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )

        # Upload feedback table
        feedback.to_sql(
            "feedback", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )

    print("✅ Data uploaded successfully with relationships intact.")

if __name__ == "__main__":
    upload_data()


## D. Read Data → PostgreSQL - (Prepare data for EDA)


In [ ]:
from sqlalchemy import create_engine
import pandas as pd


def get_db_engine():
    """Create and return a SQLAlchemy engine with proper connection string"""
    # Construct connection string from environment variables
    db_url = (
        f"postgresql://{''}:{''}@"
        f"{''}:{''}/{''}?"
    )
    return create_engine(db_url)


engine = get_db_engine()


# Query to fetch data from the tables
patients_query = "SELECT * FROM patients"
sessions_query = "SELECT * FROM sessions"
feedback_query = "SELECT * FROM feedback"

# Load data into pandas DataFrame
patients_df = pd.read_sql(patients_query, engine)
sessions_df = pd.read_sql(sessions_query, engine)
feedback_df = pd.read_sql(feedback_query, engine)




### D1- Display basic info and summary statistics for each table

In [ ]:
# Patients
print("Patients Data Overview")
patients_df.head(2)
patients_df.info()
patients_df.describe()

# Sessions
print("Sessions Data Overview")
sessions_df.head(2)
sessions_df.info()
sessions_df.describe()


# Feedback
print("Feedback Data Overview")
feedback_df.head()
feedback_df.info()
feedback_df.describe()

## E. EDA → (Exploratory Data Analysis)

Steps in EDA
 - Data Exploration
    - Head()
    - info()
    - describe()
 - Data Cleaning 
    - Identifying & Handling Missing Values
    - Identifying & Handling Duplicates
 - Data Transformation (Feature Engineering)
   - Standardizing data types (text, date, replacing values)
   - Log Transform (for highly skewed data e.g., income, prices)
   - Binning (Convert continuous variables into categorical bins e.g., age ranges)
   - Encoding (Convert categorical variables into numerical format (e.g., One-Hot, Label Encoding))
 - Data Distributions
    - Histograms for distributions of numerical features (age, bmi etc)- Also to check if the features are normally distributed or skewed (e.g., pain level or age should be somewhat evenly distributed just like we discovered in the last drop-in session).
    - Boxplots to detect outliers & handle them.
    - Pairplots to visualize relationships between multiple features.
 -  Correlation Analysis (dentify relationships between variables that may help in predictive modeling)
    - Correlation Matrix to understand feature dependencies.
    - Heatmap for visual representation of correlations.
 - Understanding Dropout Behavior
    - investigate the correlation between features and patient dropout.



#### E-2: Data Cleaning (Handling Missing Values)

In [ ]:
#5. Check for Missing Values in All Tables
print("\nMissing Values in Patients Data:")
print(patients_df.isnull().sum())

print("\nMissing Values in Sessions Data:")
print(sessions_df.isnull().sum())

print("\nMissing Values in Feedback Data:")
print(feedback_df.isnull().sum())

# 6. Clean Missing Values
# Example of filling missing values for categorical and numerical data
patients_df['chronic_cond'] = patients_df['chronic_cond'].fillna(patients_df['chronic_cond'].mode()[0])

In [ ]:
# check if null values has been properly handled
patients_df.isnull().sum()

In [ ]:
# 7. Convert Data Types ('signup_date' should be datetime)
patients_df['signup_date'] = pd.to_datetime(patients_df['signup_date'])
sessions_df['date'] = pd.to_datetime(sessions_df['date'])

### Data transformation (Feature Engineering)

In [ ]:
import numpy as np
# 8. Data Transformation: Feature Engineering (If Necessary)
# Example: Add age group column for segmentation
bins = [0, 18, 35, 55, np.inf]
labels = ['0-18', '19-35', '36-55', '55+']
patients_df['age_group'] = pd.cut(patients_df['age'], bins=bins, labels=labels)

In [ ]:
patients_df.head(2)

### Data Distribution 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Histogram for distribution of pain level
sns.histplot(sessions_df['pain_level'], kde=True)
plt.title('Distribution of Pain Level')
plt.show()

# Age distribution for patients (histogram)
sns.histplot(patients_df['age'], kde=True)
plt.title('Age Distribution')
plt.show()






